In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hindi_asr_final_only_source_target.xlsx to hindi_asr_final_only_source_target.xlsx


In [ ]:
pip install gradio transformers torch

In [ ]:
import pandas as pd

# change filename if needed
df = pd.read_excel("hindi_asr_final_only_source_target.xlsx")

df.head()

,noisy_asr_output,normalized_sentence
0,हम उसका जन्मदिन मनाया </s> <2hi>,<2hi> हमने उसका जन्मदिन मनाया। </s>
1,साउथ दिल्ली नगर निगम सख्त शॉपिंग मॉल के बाहर न...,"<2hi> साउथ दिल्ली नगर निगम सख्त, शॉपिंग मॉल के..."
2,उत्तर कोरिया अमेरिका को दी हमले की धमकी </s> <...,<2hi> उत्तर कोरिया ने अमेरिका को दी हमले की धम...
3,अगले कमरे में अक रोमन मूर्तियाँ हं </s> <2hi>,<2hi> अगले कमरे में अनेक रोमन मूर्तियाँ हैं। </s>
4,तुम टॉम को कहाँ भेज दिया </s> <2hi>,<2hi> तुम ने टॉम को कहाँ भेज दिया? </s>


In [ ]:
import pandas as pd

# Load Excel file
df = pd.read_excel("hindi_asr_final_only_source_target.xlsx")

# Assuming your columns are named 'noisy_text' and 'clean_text'
noisy_texts = df["noisy_asr_output"].tolist()
clean_texts = df["normalized_sentence"].tolist()

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Replace this with the specific IndicBART model checkpoint you want to use
model_name = "ai4bharat/indicbart"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoTokenizer

model_name = "ai4bharat/indicbart"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize inputs (noisy text)
inputs = tokenizer(
    noisy_texts,
    max_length=128,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

# Tokenize targets (clean text)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(
        clean_texts,
        max_length=128,
        padding=True,
        truncation=True,
        return_tensors="pt"
    )

# Replace pad token ids by -100 to ignore in loss
labels_ids = labels.input_ids
labels_ids[labels_ids == tokenizer.pad_token_id] = -100


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4174: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import AutoModelForSeq2SeqLM
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# The labels_ids variable is already correctly processed in cell O3-4w1I986ki.
# We will directly use the 'inputs' BatchEncoding and the processed 'labels_ids' tensor.

dataset = TensorDataset(
    inputs["input_ids"],
    inputs["attention_mask"],
    labels_ids # Use the labels_ids tensor directly from O3-4w1I986ki
)

dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(3):
    for batch_idx, batch in enumerate(dataloader):
        optimizer.zero_grad()

        # Rename 'labels' in the loop to 'labels_batch' to avoid confusion with the global 'labels' BatchEncoding
        input_ids, attention_mask, labels_batch = [x.to(device) for x in batch]

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels_batch
        )

        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch+1}, Batch {batch_idx+1}, Loss: {loss.item():.4f}")

Epoch 1, Batch 1, Loss: 4.3673
Epoch 1, Batch 2, Loss: 4.7840
Epoch 1, Batch 3, Loss: 4.0052
Epoch 1, Batch 4, Loss: 4.9123
Epoch 1, Batch 5, Loss: 4.3021
Epoch 1, Batch 6, Loss: 3.6617
Epoch 1, Batch 7, Loss: 3.7118
Epoch 1, Batch 8, Loss: 3.8559
Epoch 1, Batch 9, Loss: 4.0574
Epoch 1, Batch 10, Loss: 3.9554
Epoch 1, Batch 11, Loss: 3.9122
Epoch 1, Batch 12, Loss: 3.3800
Epoch 1, Batch 13, Loss: 3.5647
Epoch 1, Batch 14, Loss: 3.8558
Epoch 1, Batch 15, Loss: 3.1879
Epoch 1, Batch 16, Loss: 3.3955
Epoch 1, Batch 17, Loss: 3.5240
Epoch 1, Batch 18, Loss: 3.1352
Epoch 1, Batch 19, Loss: 3.2453
Epoch 1, Batch 20, Loss: 3.2676
Epoch 1, Batch 21, Loss: 3.0039
Epoch 1, Batch 22, Loss: 3.0450
Epoch 1, Batch 23, Loss: 3.4983
Epoch 1, Batch 24, Loss: 3.1791
Epoch 1, Batch 25, Loss: 3.3978
Epoch 1, Batch 26, Loss: 3.8052
Epoch 1, Batch 27, Loss: 2.8193
Epoch 1, Batch 28, Loss: 2.9963
Epoch 1, Batch 29, Loss: 2.4526
Epoch 1, Batch 30, Loss: 2.6223
Epoch 1, Batch 31, Loss: 2.6973
Epoch 1, Batch 32

In [ ]:
model.eval()

noisy_input = "एक चूहा शेर को बचाता है और अपनी दोस्ती निभाता है ,जो यह सिखाता है कि छोटे लोग भी बड़े काम आ सकते हैं </s> <2hi>"  # Example noisy input

inputs = tokenizer(noisy_input, return_tensors="pt", truncation=True)

# Move input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=50,
    num_beams=4,           # better output quality with beam search
    early_stopping=True
)

normalized_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Normalized Text:", normalized_text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Normalized Text: [CLS] एक चूहा शेर को बचाता है और अपनी दोस्ती निभाता है ,जो यह सिखाता है कि छोटे लोग भी बड़े काम आ सकते हैं। 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# SAVE MODEL TO GOOGLE DRIVE
model.save_pretrained('/content/drive/MyDrive/text_normalizer_model')
tokenizer.save_pretrained('/content/drive/MyDrive/text_normalizer_model')
print('Model saved successfully to Google Drive')

NameError: name 'model' is not defined

NameError: name 'model' is not defined

In [2]:
# LOAD MODEL FROM GOOGLE DRIVE
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/text_normalizer_model')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/text_normalizer_model')
print('Model loaded successfully from Google Drive')

Model loaded successfully from Google Drive


In [ ]:
!pip install SpeechRecognition
import gradio as gr
import torch
import speech_recognition as sr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# =======================
# LOAD MODEL
# =======================
MODEL_DIR = "/content/drive/MyDrive/text_normalizer_model"

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, local_files_only=True)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR, local_files_only=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# =======================
# TEXT NORMALIZATION
# =======================
def normalize_text(asr_text):
    if not asr_text or asr_text.strip() == "":
        return "⚠️ Please enter ASR output text."

    if not asr_text.strip().endswith("</s> <2hi>"):
        input_text = f"{asr_text.strip()} </s> <2hi>"
    else:
        input_text = asr_text.strip()

    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=128
    )

    inputs.pop("token_type_ids", None)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=64,
            num_beams=4,
            early_stopping=True
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# =======================
# AUDIO → ASR → NORMALIZE
# =======================
def normalize_speech_input(audio_path):
    r = sr.Recognizer()
    try:
        with sr.AudioFile(audio_path) as source:
            audio = r.record(source)

        asr_text = r.recognize_google(audio, language="hi-IN")
        normalized_text = normalize_text(asr_text)

        return asr_text, normalized_text

    except Exception as e:
        return "Error in ASR", str(e)

custom_css = """
html, body {
    margin: 0;
    padding: 0;
    width: 100%;
    height: 100%;
    background: linear-gradient(135deg, #0f2027, #203a43, #2c5364);
}

.gradio-container {
    max-width: 100% !important;
    width: 100% !important;
    margin: 0 !important;
}

#root, .app {
    width: 100% !important;
}

h1, h2, h3 {
    text-align: center;
    color: #ffffff;
}

textarea {
    font-size: 16px !important;
}
"""

# =======================
# GRADIO UI
# =======================
with gr.Blocks(css=custom_css, title="Hindi ASR Text Normalization") as demo:


    gr.Markdown("## 🗣️ **Hindi ASR Text Normalization Tool**")

    with gr.Tabs():

        # 🎤 VOICE TAB
        with gr.Tab("🎤 Voice → Text Normalization"):
            gr.Markdown("#### Step 1: Upload or record Hindi speech")

            audio_input = gr.Audio(
                type="filepath",
                label="Hindi Speech Input"
            )

            asr_output = gr.Textbox(
                label="Raw ASR Output",
                lines=3,
                interactive=False
            )

            normalized_output = gr.Textbox(
                label="✅ Normalized Text",
                lines=3,
                interactive=False
            )

            gr.Button("🚀 Normalize").click(
                normalize_speech_input,
                inputs=audio_input,
                outputs=[asr_output, normalized_output]
            )

        # 📝 TEXT TAB
        with gr.Tab("📝 ASR Text → Normalization"):
            gr.Markdown("#### Step 1: Paste noisy ASR output")

            text_input = gr.Textbox(
                label="Noisy ASR Text (Hindi)",
                placeholder="example: aaj mausam bahut acha hai",
                lines=4
            )

            text_output = gr.Textbox(
                label="✅ Normalized Output",
                lines=3,
                interactive=False
            )

            gr.Button(" Normalize Text").click(
                normalize_text,
                inputs=text_input,
                outputs=text_output
            )



demo.launch(debug=True)

/tmp/ipython-input-3073964433.py:101: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css, title="Hindi ASR Text Normalization") as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://436c6f71170ff85c97.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
